In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof
from datetime import datetime

In [2]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
 'all_cryptocurrencies_us','all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"],axis=1)
  y = data["suggestion"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=99)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(12,df.shape[0]-1):
        sugg = 0
        if df[each][3] > df[each][0]:
          sugg = 1
        row = [
                df[each-11][3],
                df[each-10][3],
                df[each-9][3],
                df[each-8][3],
                df[each-7][3],
                df[each-6][3],
                df[each-5][3],
                df[each-4][3],
                df[each-3][3],
                df[each-2][3],
                df[each-1][3],
                sugg
        ]
        rows.append(row)
    return rows
def row_scaler(df):
  scaler = MinMaxScaler(feature_range=(-1,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  scaled = pd.DataFrame(scaler.fit_transform(df.T).T,dtype=object,columns=["11_candels_close","10_candels_close","9_candels_close","8_candels_close","7_candels_close", "6_candels_close","5_candels_close","4_candels_close","3_candels_close","2_candels_close","1_candels_close"])
  scaled["suggestion"] = last_column
  return scaled
def column_scaler(df):
  scaler = MinMaxScaler(feature_range=(0,1))
  last_column = df.iloc[: , -1]
  df = df.drop(columns=df.columns[-1], axis=1)
  df_scaled = scaler.fit_transform(df.to_numpy())
  df_scaled = pd.DataFrame(df_scaled, columns=["11_candels_close","10_candels_close","9_candels_close","8_candels_close","7_candels_close", "6_candels_close","5_candels_close","4_candels_close","3_candels_close","2_candels_close","1_candels_close"])
  df_scaled["suggestion"] = last_column
  return df_scaled
def stick_dfs(dfs):
  dataframe = dfs[0]
  for i in range(1,len(dfs)):
     dataframe = pd.concat([dataframe,dfs[i]], ignore_index = True)
  dataframe = dataframe.dropna()
  dataframe = dataframe.astype(float)
  return dataframe
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      df = np.array(df)
      df = extract_data(df)
      df = pd.DataFrame(df,columns = ["11_candels_close","10_candels_close","9_candels_close","8_candels_close","7_candels_close", "6_candels_close","5_candels_close","4_candels_close","3_candels_close","2_candels_close","1_candels_close","suggestion"])
      df = row_scaler(df)
      #df = column_scaler(df)
      fixed_dfs.append(df)
   df = stick_dfs(fixed_dfs)
   return df   

In [11]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]
#df = pd.read_parquet("/content/drive/MyDrive/Colab Files/250_crypto_1d_scaled_yf.parquet")

symbols = get_crypto_syms()
dfs = []
for symbol in symbols[:50]:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 13:
          dfs.append(df)
data = process(dfs)
now = datetime.now()
current_time = now.strftime("%H%M%S")
data.to_csv(f"{current_time}.csv")
xTrain, xTest, yTrain, yTest = spliting(data)
data

(50248, 11) (50248,)
(5584, 11) (5584,)


,11_candels_close,10_candels_close,9_candels_close,8_candels_close,7_candels_close,6_candels_close,5_candels_close,4_candels_close,3_candels_close,2_candels_close,1_candels_close,suggestion
0,0.612660,-0.398908,0.082512,-0.261559,-0.147892,1.000000,0.570517,0.173623,-0.070329,-0.237707,-1.000000,0.0
1,-0.359161,0.108580,-0.225714,-0.115277,1.000000,0.582720,0.197103,-0.039918,-0.202540,-0.943174,-1.000000,1.0
2,0.108580,-0.225714,-0.115277,1.000000,0.582720,0.197103,-0.039918,-0.202540,-0.943174,-1.000000,-0.619488,0.0
3,-0.225714,-0.115277,1.000000,0.582720,0.197103,-0.039918,-0.202540,-0.943174,-1.000000,-0.619488,-0.729859,0.0
4,-0.108022,1.000000,0.585435,0.202326,-0.033153,-0.194717,-0.930532,-0.986989,-0.608952,-0.718605,-1.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
55827,1.000000,0.279737,0.278237,0.076891,0.405457,0.113711,-0.214810,-0.920008,-0.617091,-1.000000,-0.932090,0.0
55828,0.836637,0.834688,0.573057,1.000000,0.620901,0.194017,-0.722325,-0.328711,-0.826268,-0.738025,-1.000000,0.0
55829,0.899301,0.739930,1.000000,0.769074,0.509040,-0.049146,0.190622,-0.112462,-0.058709,-0.218290,-1.000000,1.0
55830,0.739930,1.000000,0.769074,0.509040,-0.049146,0.190622,-0.112462,-0.058709,-0.218290,-1.000000,-0.895149,1.0


In [12]:
model = Sequential()
adam = tf.keras.optimizers.Adam(
    learning_rate=0.002)

model.add(Dense(1024, activation='relu', input_shape=(xTrain.shape[1],)))
#model.add(Dense(4096, activation='relu'))
#model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(512, activation='relu'))


model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 1024)              12288     
                                                                 
 dense_21 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_22 (Dense)            (None, 750)               768750    
                                                                 
 dense_23 (Dense)            (None, 512)               384512    
                                                                 
 dense_24 (Dense)            (None, 1)                 513       
                                                                 
Total params: 2,215,663
Trainable params: 2,215,663
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(xTrain,yTrain,epochs=300,batch_size=64,validation_data=(xTest,yTest))
model.save(f"{model.evaluate(xTest,yTest)}.h5")

Epoch 1/300
786/786 [==============================] - 6s 7ms/step - loss: 0.6901 - accuracy: 0.5294 - val_loss: 0.6920 - val_accuracy: 0.5158
Epoch 2/300
786/786 [==============================] - 5s 7ms/step - loss: 0.6876 - accuracy: 0.5402 - val_loss: 0.6875 - val_accuracy: 0.5247
Epoch 3/300
786/786 [==============================] - 5s 7ms/step - loss: 0.6845 - accuracy: 0.5455 - val_loss: 0.6861 - val_accuracy: 0.5458
Epoch 4/300
786/786 [==============================] - 5s 7ms/step - loss: 0.6827 - accuracy: 0.5537 - val_loss: 0.6846 - val_accuracy: 0.5396
Epoch 5/300
786/786 [==============================] - 5s 7ms/step - loss: 0.6809 - accuracy: 0.5568 - val_loss: 0.6896 - val_accuracy: 0.5466
Epoch 6/300
786/786 [==============================] - 5s 7ms/step - loss: 0.6780 - accuracy: 0.5636 - val_loss: 0.6830 - val_accuracy: 0.5562
Epoch 7/300
786/786 [==============================] - 5s 7ms/step - loss: 0.6754 - accuracy: 0.5680 - val_loss: 0.6811 - val_accuracy: 0.5480

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")

In [ ]:
import os
files = os.listdir("/content/gdrive/MyDrive/Colab Files/")

data = pd.DataFrame()
for file in files:
  address = f"/content/gdrive/MyDrive/Colab Files/{file}"
  df = pd.read_csv(address)
  data = pd.concat([data,df])

In [ ]:
symbols = read_syms_from_txt()
len(symbols)
pieces = 15
new_arrays = np.array_split(symbols, pieces)
for i in new_arrays:
  dfs = []
  for x in i:
        data = yf.download(x,period="MAX",interval="1d",progress=False)
        if data.empty :
             print("Passing...")
        else:
               dfs.append(data)
  ndfs = dfs
  dfs = []
  for df in ndfs:
        if df.shape[0] > 12:
          dfs.append(df)
  data = process(dfs)
  data.to_csv(f"{x}.csv")         

In [ ]:
address = "/content/gdrive/MyDrive/1d_10pastcandle_newversion.csv"
from google.colab import drive
drive.mount("/content/gdrive/")
data = pd.read_csv(address)


Mounted at /content/gdrive/


NameError: ignored

In [ ]:
data

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,sugg
0,0,0.723113,0.000000,0.344140,0.098183,0.179437,1.000000,0.692987,0.409270,0.234882,0.115234,0.0
1,1,0.300546,0.541256,0.369221,0.426054,1.000000,0.785258,0.586811,0.464835,0.381146,0.000000,0.0
2,2,0.554290,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,1.0
3,3,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.0
4,4,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.135070,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1761617,1761617,0.000000,0.110425,0.174078,0.177459,0.408812,0.400589,0.305815,0.610451,0.722925,1.000000,1.0
1761618,1761618,0.000000,0.060724,0.063949,0.284657,0.276813,0.186399,0.477018,0.584317,0.848643,1.000000,1.0
1761619,1761619,0.000000,0.003369,0.233901,0.225708,0.131269,0.434825,0.546900,0.822992,0.981087,1.000000,0.0
1761620,1761620,0.000000,0.231311,0.223090,0.128333,0.432914,0.545369,0.822394,0.981023,1.000000,0.755756,0.0


In [ ]:
data = data.drop(["Unnamed: 0"],axis=1)
xTrain, xTest, yTrain, yTest = spliting(data)
data

(1585459, 10) (1585459,)
(176163, 10) (176163,)


,0,1,2,3,4,5,6,7,8,9,sugg
0,0.723113,0.000000,0.344140,0.098183,0.179437,1.000000,0.692987,0.409270,0.234882,0.115234,0.0
1,0.300546,0.541256,0.369221,0.426054,1.000000,0.785258,0.586811,0.464835,0.381146,0.000000,0.0
2,0.554290,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,1.0
3,0.387143,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.0
4,0.442361,1.000000,0.791360,0.598551,0.480041,0.398730,0.028413,0.000000,0.190256,0.135070,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1761617,0.000000,0.110425,0.174078,0.177459,0.408812,0.400589,0.305815,0.610451,0.722925,1.000000,1.0
1761618,0.000000,0.060724,0.063949,0.284657,0.276813,0.186399,0.477018,0.584317,0.848643,1.000000,1.0
1761619,0.000000,0.003369,0.233901,0.225708,0.131269,0.434825,0.546900,0.822992,0.981087,1.000000,0.0
1761620,0.000000,0.231311,0.223090,0.128333,0.432914,0.545369,0.822394,0.981023,1.000000,0.755756,0.0
